# PART A: TRAINING

### Instruction 1: Train a 2-layer LSTM on the PTB data set. Again: I want TWO layers, not one. 

### Instruction 2: your code should display something like this during training:

* epoch 0: &nbsp; &nbsp; time= &nbsp; &nbsp;  lr= &nbsp;&nbsp; train: exp(loss)= 
* test: exp(loss) =




* epoch 1: &nbsp; &nbsp; time= &nbsp; &nbsp;  lr= &nbsp;&nbsp; train: exp(loss)= 
* test: exp(loss) =




* epoch 2: &nbsp; &nbsp; time= &nbsp; &nbsp;  lr= &nbsp;&nbsp; train: exp(loss)= 
* test: exp(loss) =





### Instruction 3: Use a learning rate schedule. It is very tricky to find a good schedule. I recommend the following: keep the same learning rate for a few epochs at the beginning of the training (maybe during 2, 3, 4, 5 or 6 epochs, experiment and see what works the best). After this first phase, divide the learning rate by a relatively large factor at the end of every epoch.

### Instruction 4: You need to obtain a perplexity (perplexity=exp(loss)) less than 110 on the TEST set in order to get full credit. Going below 115 will give you partial credit. 

### Instruction 5: Answer the questions below to describe your network, and how you have tuned the hyperparameters.

### Instruction 6: Read very carefully intructions 1 to 5 above!

# ----------------------------------------------------------------------------------

### What is the number of parameters in your network?

2653200 (2.65 million) parameters in this neural network

### Describe the learning schedule that you have used.

I began with a learning rate of 6.66 then at the 4th epoch, this value is divided by (4*n) where n is initialized is 0.5. After each epoch from here, the value of n is increased by 0.5, and no longer decreases the learning rate if it is equal to or below 0.06.

### Described what hyperparameters (batchsize, learning rate, hiddensize, etc...) you tried. Which of these hyperparameters was the most important to correctly tune? Which one make the most difference?


1.

7454800 (7.45 million) parameters in this neural network
lr = 4
hiddenSize = 300
bs = 20
seqLength = 25

error = 113.13

2.

7454800 (7.45 million) parameters in this neural network
lr = 4
hiddenSize = 300
bs = 20
seqLength = 40

error  = 128.15

3. 



2653200 (2.65 million) parameters in this neural network
lr = 6.66
hidden_size = 200
bs = 20
seqLength = 20

The most important to correctly tune was the learning rate. It required a large value for the first few epochs, then this is eventually decreased so that the network does not increase in error or stray too far from where it currently is in terms of perplexity.




# ----------------------------------------------------------------------------------

### Write your code below

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import math
import utils
import time

In [2]:
device = torch.device("cuda")
train_data  =  torch.load('../../data/ptb/train_data.pt')
test_data   =  torch.load('../../data/ptb/test_data.pt')

In [3]:
class twoLayer_LSTM(nn.Module):
    
    def __init__(self, vocab_size, hidden_size,layers):
        super().__init__()
        
        self.emb_layer = nn.Embedding(vocab_size, hidden_size)
        self.rec_layer = nn.LSTM(hidden_size, hidden_size, num_layers=layers)
        self.lin_layer = nn.Linear(hidden_size, vocab_size)

    def forward(self, word_seq, h_init, c_init):
        g_seq = self.emb_layer(word_seq)  
        h_seq, (h_last, c_last) = self.rec_layer(g_seq , (h_init, c_init))
        score_seq = self.lin_layer(h_seq)
        
        return score_seq, (h_last, c_last)
    
def eval_on_test_set():

    running_loss=0
    num_batches=0    
    
    with torch.no_grad():
       
        h = torch.zeros(layers, bs, hidden_size)
        c = torch.zeros(layers, bs, hidden_size)

        h = h.to(device)
        c = c.to(device)


        for count in range( 0 , testSize-seq_length ,  seq_length) :

            minibatch_data =  test_data[ count   : count+seq_length   ]
            minibatch_label = test_data[ count+1 : count+seq_length+1 ]

            minibatch_data = minibatch_data.to(device)
            minibatch_label = minibatch_label.to(device)

            scores, (h,c)  = net(minibatch_data, h, c)

            minibatch_label =   minibatch_label.view(  bs*seq_length ) 
            scores          =   scores.view(  bs*seq_length , vocab_size)

            loss = criterion(  scores ,  minibatch_label )    

            h = h.detach()
            c = c.detach()

            running_loss += loss.item()
            num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )

In [4]:
hidden_size = 200
vocab_size = 10000
layers = 2

net = twoLayer_LSTM(vocab_size,hidden_size,layers)
net.emb_layer.weight.data.uniform_(-0.1, 0.1)
net.lin_layer.weight = net.emb_layer.weight
utils.display_num_param(net)
print(net)
net = net.to(device)
criterion = nn.CrossEntropyLoss()
my_lr = 6.66
trainSize = len(train_data)
testSize = len(test_data)
bs = 20
seq_length = 20
optimizer = optim.SGD(net.parameters(), lr=my_lr) 

There are 2653200 (2.65 million) parameters in this neural network
twoLayer_LSTM(
  (emb_layer): Embedding(10000, 200)
  (rec_layer): LSTM(200, 200, num_layers=2)
  (lin_layer): Linear(in_features=200, out_features=10000, bias=True)
)


In [5]:
start=time.time()
n = 0.5
for epoch in range(15):
    if epoch >= 4: 
        if my_lr > 0.06 :
            my_lr = my_lr/(4*n)
        n += 0.5
        optimizer = optim.SGD(net.parameters(), lr=my_lr) 
            
    # set the running quantities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h to be the zero vector
    h = torch.zeros(layers, bs, hidden_size)
    c = torch.zeros(layers, bs, hidden_size)

    # send it to the gpu    
    h = h.to(device)
    c = c.to(device)

    for count in range( 0 , trainSize-seq_length ,  seq_length):    
            
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
                
        # send them to the gpu
        minibatch_data = minibatch_data.to(device)
        minibatch_label = minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h = h.detach()
        c = c.detach()
        h = h.requires_grad_()
        c = c.requires_grad_()
        # forward the minibatch through the net 
        scores, (h,c)  = net(minibatch_data, h, c)
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
                          
    # epoch finished:  compute and display stats for the full training set

    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    print('\nepoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))    # compute error on the test set at end of each epoch
    eval_on_test_set() 
            
eval_on_test_set() # eval on test set one last time
print(" ")      


epoch= 0 	 time= 8.911894083023071 	 lr= 6.66 	 exp(loss)= 255.09866260314092
test: exp(loss) =  175.1922920333654

epoch= 1 	 time= 18.10270357131958 	 lr= 6.66 	 exp(loss)= 132.3375802207384
test: exp(loss) =  142.40243987277162

epoch= 2 	 time= 27.280725717544556 	 lr= 6.66 	 exp(loss)= 106.68706518940175
test: exp(loss) =  132.57446797828698

epoch= 3 	 time= 36.497321367263794 	 lr= 6.66 	 exp(loss)= 93.61326416323317
test: exp(loss) =  128.6558435852042

epoch= 4 	 time= 45.70159149169922 	 lr= 3.33 	 exp(loss)= 72.59998400072656
test: exp(loss) =  113.66611998641604

epoch= 5 	 time= 54.8425018787384 	 lr= 0.8325 	 exp(loss)= 58.00795317168163
test: exp(loss) =  108.37556227062005

epoch= 6 	 time= 63.97396636009216 	 lr= 0.13875 	 exp(loss)= 52.7321277455055
test: exp(loss) =  106.81076586069734

epoch= 7 	 time= 73.17681503295898 	 lr= 0.01734375 	 exp(loss)= 51.48149420198122
test: exp(loss) =  105.81156824423367

epoch= 8 	 time= 82.35340166091919 	 lr= 0.01734375 	 exp(lo

# ----------------------------------------------------------------------------------

# ----------------------------------------------------------------------------------

# PART B: INFERENCE

### Write 4 codes in 4 separate cells that do the following
* In each cell you have a sentence taken from ptb test set. Use utils.text2tensor() to convert this sentence into a LongTensor. 
* Feed the sentence to the network
* Your network should compute a probability vector over the full vocabulary of 10,000 words. This vector tells you which words are likely to come next. Obviously we can not display this huge probability vector, so we are just going to display the 30 most likely words according to the network. To do this feed the probability vector with 10,000 entries to the function utils.show_most_likely_words(). This function is going to sort the probabilities and show you the 30 highest probabilities as well as the corresponding words.

In [6]:
sentence = "prices averaging roughly $ N a barrel higher in the third"

h = torch.zeros(layers, bs, hidden_size)
c = torch.zeros(layers, bs, hidden_size)
h = h.to(device)
c = c.to(device)

data = utils.text2tensor(sentence)
seqLength = len(data)
data = data.view(seqLength,-1)
empty = torch.zeros(seqLength,19).type(torch.LongTensor)
data = torch.cat((data,empty),dim=1)
data = data.to(device)
scores, (h,c)  = net(data,h,c)
scores = scores[seqLength-1,0,:]
p = F.softmax(scores.view(1,vocab_size),dim=1)
print(sentence, '... \n')
utils.show_most_likely_words(p)  

prices averaging roughly $ N a barrel higher in the third ... 

22.6%	 quarter
10.7%	 period
9.8%	 consecutive
8.2%	 or
3.9%	 game
2.4%	 <eos>
2.4%	 trading
1.9%	 half
1.9%	 day
1.8%	 and
1.7%	 of
1.1%	 since
1.1%	 place
1.1%	 in
1.1%	 <unk>
0.9%	 straight
0.8%	 year
0.7%	 to
0.7%	 hour
0.7%	 quarters
0.7%	 session
0.6%	 months
0.5%	 market
0.5%	 floor
0.5%	 weeks
0.5%	 month
0.4%	 minutes
0.4%	 full
0.4%	 near
0.4%	 times


In [7]:
sentence = "i think my line has been very consistent mrs. hills said at a news"

h = torch.zeros(layers, bs, hidden_size)
c = torch.zeros(layers, bs, hidden_size)
h = h.to(device)
c = c.to(device)

data = utils.text2tensor(sentence)
seqLength = len(data)
data = data.view(seqLength,-1)
empty = torch.zeros(seqLength,19).type(torch.LongTensor)
data = torch.cat((data,empty),dim=1)
data = data.to(device)
scores, (h,c)  = net(data,h,c)
scores = scores[seqLength-1,0,:]
p = F.softmax(scores.view(1,vocab_size),dim=1)
print(sentence, '... \n')
utils.show_most_likely_words(p)  

i think my line has been very consistent mrs. hills said at a news ... 

97.2%	 conference
0.2%	 panel
0.2%	 research
0.1%	 <unk>
0.1%	 university
0.1%	 's
0.1%	 study
0.1%	 time
0.1%	 agency
0.1%	 institute
0.0%	 firm
0.0%	 meeting
0.0%	 committee
0.0%	 ohio
0.0%	 moment
0.0%	 earlier
0.0%	 spokesman
0.0%	 official
0.0%	 trial
0.0%	 statement
0.0%	 newsletter
0.0%	 exchange
0.0%	 dinner
0.0%	 program
0.0%	 close
0.0%	 lawyer
0.0%	 ad
0.0%	 club
0.0%	 opening
0.0%	 hearing


In [8]:
sentence = "this appears particularly true at gm which had strong sales in the"
h = torch.zeros(layers, bs, hidden_size)
c = torch.zeros(layers, bs, hidden_size)
h = h.to(device)
c = c.to(device)

data = utils.text2tensor(sentence)
seqLength = len(data)
data = data.view(seqLength,-1)
empty = torch.zeros(seqLength,19).type(torch.LongTensor)
data = torch.cat((data,empty),dim=1)
data = data.to(device)
scores, (h,c)  = net(data,h,c)
scores = scores[seqLength-1,0,:]
p = F.softmax(scores.view(1,vocab_size),dim=1)
print(sentence, '... \n')
utils.show_most_likely_words(p)  

this appears particularly true at gm which had strong sales in the ... 

13.1%	 u.s.
6.1%	 past
3.6%	 <unk>
3.2%	 next
3.2%	 first
2.9%	 company
2.6%	 N
2.4%	 second
2.3%	 third
2.1%	 industry
2.1%	 fourth
1.8%	 world
1.2%	 future
1.0%	 quarter
1.0%	 year
1.0%	 stock
0.9%	 current
0.8%	 market
0.7%	 european
0.6%	 1990s
0.6%	 markets
0.6%	 wake
0.6%	 fiscal
0.6%	 business
0.5%	 $
0.5%	 region
0.5%	 new
0.5%	 years
0.5%	 same
0.5%	 full


In [9]:
sentence = "some analysts expect oil prices to remain relatively"
h = torch.zeros(layers, bs, hidden_size)
c = torch.zeros(layers, bs, hidden_size)
h = h.to(device)
c = c.to(device)

data = utils.text2tensor(sentence)
seqLength = len(data)
data = data.view(seqLength,-1)
empty = torch.zeros(seqLength,19).type(torch.LongTensor)
data = torch.cat((data,empty),dim=1)
data = data.to(device)
scores, (h,c)  = net(data,h,c)
scores = scores[seqLength-1,0,:]
p = F.softmax(scores.view(1,vocab_size),dim=1)
print(sentence, '... \n')
utils.show_most_likely_words(p)  

some analysts expect oil prices to remain relatively ... 

18.0%	 high
11.8%	 low
9.2%	 slow
7.1%	 strong
4.7%	 <unk>
3.8%	 good
3.6%	 flat
2.6%	 operational
2.5%	 small
2.3%	 narrow
1.7%	 competitive
1.6%	 higher
1.4%	 little
1.4%	 thin
1.4%	 different
1.2%	 weak
1.0%	 diluted
1.0%	 heavy
1.0%	 tight
1.0%	 large
0.9%	 lower
0.7%	 stable
0.6%	 positive
0.5%	 cheap
0.5%	 steady
0.4%	 complex
0.4%	 weaker
0.4%	 healthy
0.4%	 significant
0.4%	 light


### Make a 3 sentences of your own (they should related to economy) and see what the network predict.

In [10]:
sentence = "The job market seems"

h = torch.zeros(layers, bs, hidden_size)
c = torch.zeros(layers, bs, hidden_size)
h = h.to(device)
c = c.to(device)

data = utils.text2tensor(sentence)
seqLength = len(data)
data = data.view(seqLength,-1)
empty = torch.zeros(seqLength,19).type(torch.LongTensor)
data = torch.cat((data,empty),dim=1)
data = data.to(device)
scores, (h,c)  = net(data,h,c)
scores = scores[seqLength-1,0,:]
p = F.softmax(scores.view(1,vocab_size),dim=1)
print(sentence, '... \n')
utils.show_most_likely_words(p)   

The job market seems ... 

51.0%	 to
4.2%	 likely
3.0%	 so
2.9%	 <unk>
1.6%	 like
1.5%	 a
1.4%	 not
1.3%	 more
1.0%	 that
0.9%	 better
0.8%	 as
0.7%	 in
0.7%	 beyond
0.7%	 out
0.6%	 on
0.5%	 no
0.5%	 about
0.4%	 all
0.4%	 good
0.4%	 for
0.4%	 nothing
0.4%	 less
0.4%	 too
0.4%	 far
0.4%	 needed
0.3%	 at
0.3%	 the
0.3%	 going
0.3%	 still
0.3%	 an


In [11]:
sentence = "The economy is doing"

h = torch.zeros(layers, bs, hidden_size)
c = torch.zeros(layers, bs, hidden_size)
h = h.to(device)
c = c.to(device)

data = utils.text2tensor(sentence)
seqLength = len(data)
data = data.view(seqLength,-1)
empty = torch.zeros(seqLength,19).type(torch.LongTensor)
data = torch.cat((data,empty),dim=1)
data = data.to(device)
scores, (h,c)  = net(data,h,c)
scores = scores[seqLength-1,0,:]
p = F.softmax(scores.view(1,vocab_size),dim=1)
print(sentence, '... \n')
utils.show_most_likely_words(p)  

The economy is doing ... 

7.4%	 <eos>
7.1%	 a
6.2%	 <unk>
5.0%	 the
4.2%	 much
2.8%	 it
2.8%	 well
2.6%	 so
2.1%	 this
2.0%	 more
1.9%	 that
1.7%	 little
1.3%	 in
1.3%	 something
1.3%	 an
1.1%	 good
1.0%	 high
1.0%	 as
0.9%	 business
0.9%	 at
0.7%	 when
0.7%	 if
0.7%	 anything
0.6%	 bad
0.6%	 any
0.6%	 what
0.6%	 strong
0.6%	 very
0.6%	 some
0.5%	 you


In [12]:
sentence = "Right now I want to buy a house and buying a house is"

h = torch.zeros(layers, bs, hidden_size)
c = torch.zeros(layers, bs, hidden_size)
h = h.to(device)
c = c.to(device)

data = utils.text2tensor(sentence)
seqLength = len(data)
data = data.view(seqLength,-1)
empty = torch.zeros(seqLength,19).type(torch.LongTensor)
data = torch.cat((data,empty),dim=1)
data = data.to(device)
scores, (h,c)  = net(data,h,c)
scores = scores[seqLength-1,0,:]
p = F.softmax(scores.view(1,vocab_size),dim=1)
print(sentence, '... \n')
utils.show_most_likely_words(p)  

Right now I want to buy a house and buying a house is ... 

10.0%	 n't
6.0%	 <unk>
5.3%	 not
3.0%	 a
2.6%	 to
2.2%	 the
2.1%	 being
1.3%	 that
1.3%	 going
1.2%	 in
1.2%	 likely
1.1%	 expected
1.1%	 just
1.1%	 very
1.0%	 on
1.0%	 getting
0.9%	 more
0.8%	 taking
0.7%	 trying
0.7%	 now
0.6%	 <eos>
0.6%	 making
0.6%	 still
0.6%	 doing
0.6%	 considering
0.6%	 seeking
0.6%	 an
0.5%	 one
0.5%	 necessary
0.5%	 too
